#### In this notebook, we make predictions of LC, HALC, CS (made_claim), using external models built from other notebooks
- RUN ORDER 5/5

- Prediction Methodology: Two-step and chained modeling approach
  - We use a two-step modeling approach
    - We made a binary classification model to predict if LC is 0 or non 0
      - Use SMOTE to balance the unbalanced data
        - first, must convert LC to is_LC, where is_LC is a binary feature where 1 if LC != 0, else 0
      - We train it all the data in insurance_train.csv for all features except HALC and made_claim (CS)
    - We then parse the data in insurance_train.csv where LC is not equal to 0
      - We build a random forest regressor based off of the parse data.
      - We predict the actual value of LC if LC is not 0
    - Doing this helps us handle the imbalanced data plus improves the accuracy on the non-zero classes, yet still predicts the majority class properly without harming the predictions of the minority class.
  - Chained modeling approach:
    - after we predict LC, we can then predict HALC (again, we don't use made_claim CS)
    - We predict HALC with a random forest regressor fitted by the data where LC != 0
    - If LC is 0, we assume HALC is also 0
      - This assumption makes sense, as HALC = LC * Ratio
    - Finally, we predict made_claim CS, using all of the data, where made_claim is a binary feature. 

- Rationale:
  - Converting LC to is_LC is required for SMOTE as SMOTE only is useful for classification. We SMOTE because there is heavy class imbalance in LC
  - Building regressor for when LC does not equal 0 prevents the model from being skewed towards predicting 0; will predict an actual number while maintaining integrity of 0's
    - Using un-parsed data to predict LC will have all instances not equal to 0, which is not correct
  - Chained modeling: We predict HALC after LC since HALC is dependent on LC
  - Data leakage prevention: we do not predict made_claim CS during LC and HALC predictions, since made_claim CS is dependent on LC and HALC.
    - This should improve accuracy, especially if LC and HALC are strong predictors

- Potential improvements:
  - Tuning classification thresholds from the default 0.5 in is_LC
  - HALC: instead of predicting HALC with LC, we predict the ratio of number of claims filed (X.18), then multiple hat(X.18) and hat(LC)
    - Doing this may prevent conpounding errors, as hat(HALC) is currently heavily dependent on hat(LC)

- X.1 --> Internal Identification
- X.2 --> pol_start: start date of the policy holder's contract (DD/MM/YYYY)
- X.3 --> last_renewal: date of last contract renewal (DD/MM/YYYY)
- X.4 --> next_renewal: date of next contract renewal (DD/MM/YYYY)
- X.5 --> DOB: date of birth of the insured declared in policy
- X.6 --> DL_issuance: Date of license insuance\
- X.10 -->  max_policies: Max number of policies that insured has ever had in force.
- X.11 -->  max_products: Max number of products that insured has simultaneously held at any given point.
- X.12  -->  canceled_policies: Number of policies canceled or terminated for nonpayment in the current year.
-  X.13  -->  is_halfyearly: Last payment method of the reference policy (1: half-yearly, 0: annual).
- X.14  --> net_premium: Net premium amount associated with the policy during the current year.
- X.15  --> claim_cost: Total cost of claims for the insurance policy during the current year.
- X.16  --> claim_count: Total number of claims incurred for the insurance policy during the current year.
- X.17  --> total_claims: Total number of claims filed throughout the entire duration of the policy.
- X.18  --> claim_ratio: Ratio of the number of claims filed to the total duration (years) of the policy in force.
- X.19  --> risk_type: Type of risk (1: motorbikes, 2: vans, 3: passenger cars, 4: agricultural vehicles).
- X.20 --> is_urban (1=urban, 0=rural): 0 for rural, 1 for urban (more than 30,000 inhabitants).
- X.21  --> is_multidriver: 1 if multiple regular drivers are declared, 0 if only one driver is declared.
- X.22  --> regis_year: Year of vehicle registration (YYYY).
- X.23  --> horsepower: Vehicle power measured in horsepower.
- X.24  --> cylinder_cap: Cylinder capacity of the vehicle.
- X.25  --> market_value: Market value of the vehicle as of 31/12/2019.
- X.26  --> door_count: Number of vehicle doors.
- X.27  --> is_petrol (1= petrol, 0 =Diesel): Energy source used to power the vehicle (P: Petrol, D: Diesel).
- X.28  --> vehicle_weight: Vehicle weight in kilograms.


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
import joblib
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv("data/insurance_test.csv")

In [8]:
df.rename(columns={'X.27': 'is_petrol'}, inplace=True)

In [9]:
df['is_petrol'] = df['is_petrol'].map({'P': 1, 'D': 0})

In [10]:
df = df.rename(columns={
    'X.2': 'pol_start', 
    'X.3': 'last_renewal',
    'X.4': 'next_renewal',
    'X.5': 'DOB',
    'X.6': 'license_issue_date',
    'X.7': 'is_channel_broker',
    'X.8': 'total_pol_year',
    'X.9': 'total_pol_held',
    'X.10': 'max_policies',
    'X.11': 'max_products',
    'X.12': 'canceled_policies',
    'X.13': 'is_halfyearly',
    'X.14': 'net_premium',
    'X.19': 'risk_type',
    'X.20': 'is_urban',
    'X.21': 'is_multidriver',
    'X.22': 'regis_year',
    'X.23': 'horsepower',
    'X.24': 'cylinder_cap',
    'X.25': 'market_value',
    'X.26': 'door_count',
    'X.28': 'vehicle_weight',
})

In [11]:
dates_to_convert = ['pol_start', 'last_renewal', 'next_renewal', 'license_issue_date']

In [12]:
for i in dates_to_convert:
    df[i] = pd.to_datetime(df[i], format='%d/%m/%Y')
    df[f'{i}_day'] = df[i].dt.day
    df[f'{i}_month'] = df[i].dt.month
    df[f'{i}_year'] = df[i].dt.year

In [13]:
df = df.drop(columns=dates_to_convert)

In [14]:
df['DOB'] = pd.to_datetime(df['DOB'], format='%d/%m/%Y')

In [15]:
reference_date = pd.to_datetime('31/12/2019', format='%d/%m/%Y')

In [16]:
df['Age'] = (reference_date - df['DOB']).dt.days // 365

In [17]:
df = df.drop(columns='DOB')

In [18]:
df['is_youngin'] = (df['Age'] <= 24).astype(int)
df['is_adult'] = ((df['Age'] >= 25) & (df['Age'] <= 39)).astype(int)
df['is_middleaged'] = ((df['Age'] >= 40) & (df['Age'] <= 64)).astype(int)
df['is_old'] = (df['Age'] >= 65).astype(int)

In [19]:
df = df.drop(columns='Age')

In [20]:
df.columns

Index(['is_channel_broker', 'total_pol_year', 'total_pol_held', 'max_policies',
       'max_products', 'canceled_policies', 'is_halfyearly', 'net_premium',
       'risk_type', 'is_urban', 'is_multidriver', 'regis_year', 'horsepower',
       'cylinder_cap', 'market_value', 'door_count', 'is_petrol',
       'vehicle_weight', 'pol_start_day', 'pol_start_month', 'pol_start_year',
       'last_renewal_day', 'last_renewal_month', 'last_renewal_year',
       'next_renewal_day', 'next_renewal_month', 'next_renewal_year',
       'license_issue_date_day', 'license_issue_date_month',
       'license_issue_date_year', 'is_youngin', 'is_adult', 'is_middleaged',
       'is_old'],
      dtype='object')

i have to apply the same thing did for the training df to the testing df. but for training, i have to include the logistic imputation for is_petrol NaNs. the testing data set does not have any NaNs. 
- We fit the model with X_train, y_train
- then we predict with df_unseen
- We take the values predicted from df_unseen, and if the value is a 1, we fit that instance in the other model (built by katie) to predict a continuous value

In [22]:
# load model that predicts if an instance is a 0 or 1
# model from: _project_insurance_3.ipynb
rf = joblib.load('temp.pkl')

In [23]:
# y_pred = predictions where instance is 0 or 1
y_pred = rf.predict(df)
y_proba = rf.predict_proba(df)[:, 1]

In [24]:
len(y_pred[y_pred == 0])

14882

In [25]:
len(y_pred[y_pred == 1])

905

In [26]:
df.columns

Index(['is_channel_broker', 'total_pol_year', 'total_pol_held', 'max_policies',
       'max_products', 'canceled_policies', 'is_halfyearly', 'net_premium',
       'risk_type', 'is_urban', 'is_multidriver', 'regis_year', 'horsepower',
       'cylinder_cap', 'market_value', 'door_count', 'is_petrol',
       'vehicle_weight', 'pol_start_day', 'pol_start_month', 'pol_start_year',
       'last_renewal_day', 'last_renewal_month', 'last_renewal_year',
       'next_renewal_day', 'next_renewal_month', 'next_renewal_year',
       'license_issue_date_day', 'license_issue_date_month',
       'license_issue_date_year', 'is_youngin', 'is_adult', 'is_middleaged',
       'is_old'],
      dtype='object')

my thought process: get rows where is_LC is 1. fit a linear regression model to get continuous values. fit those continuous values to predict HALC. then use all of that information to predict if they made a claim or not

In [28]:
len(y_pred)

15787

In [29]:
len(df)

15787

In [30]:
# take binary predictions and add them into the data set
df['is_LC'] = y_pred

Now that we have the instances where LC is 0, let's find out what the values are if LC is not 0

In [32]:
# parse data where LC is not 0
df_is_LC = df[df['is_LC'] == 1]

In [33]:
# now, we need to fit a tree based model
# predict value of LC in df_is_LC 

In [34]:
# next steps, build a tree based model from training data. 
# use that model on df_is_LC

In [35]:
# load the model that predicts the value of LC if LC is not 0
# model from: building_RF_for_LC.ipynb
another_rf = joblib.load('predict_LC.pkl')

In [36]:
df_for_pred = df_is_LC.drop(columns=['is_LC'])

In [37]:
y_pred = another_rf.predict(df_for_pred)
# y_proba = another_rf.predict_proba(df_for_pred)[:, 1]

In [38]:
# these are the predicted values of LC if it is not 0
y_pred[0:4] # truncated

array([537.00004167, 303.00840333, 347.30058   , 313.69362833])

In [39]:
# now i need to put this data back in the original dataset with the 0s, and ensure they are in order

In [40]:
temp = df['is_LC'] == 1
df.loc[temp, 'is_LC'] = y_pred

In [41]:
min(df[df['is_LC'] != 0]['is_LC'])

109.52390436813194

In [42]:
# we are renaming the column is_LC BACK TO LC because it now contains 0's and actual numbers
df = df.rename(columns={'is_LC': 'LC'})

In [43]:
# df['LC'].to_csv("LC", index=False)

In [44]:
# Now that we have LC, we can use it to help predict HALC
# NOTE: We could always predict X.18 and just multiply that by our predicted LC, but we opted to just directly predict HALC
# model from: building_RF_for_HALC.ipynb
rf_HALC = joblib.load('predict_HALC.pkl')

In [45]:
# extract where LC does not equal 0
# again, we only want to make predictions where LC is not 0. 
# why is that? because if LC is 0, we assume that HALC must also be 0. 
# including the data where LC is already 0 is noisy and redundant since we only care about instances where LC (and by extention HALC) does not equal 0
df_has_LC = df[df['LC'] != 0]

In [46]:
df_has_LC

,is_channel_broker,total_pol_year,total_pol_held,max_policies,max_products,canceled_policies,is_halfyearly,net_premium,risk_type,is_urban,...,next_renewal_month,next_renewal_year,license_issue_date_day,license_issue_date_month,license_issue_date_year,is_youngin,is_adult,is_middleaged,is_old,LC
16,1,3,1,1,1,0,0,159.66,1,0,...,2,2017,21,3,1991,0,0,1,0,537.000042
34,0,2,2,2,1,0,0,376.22,2,0,...,1,2019,6,8,2009,0,1,0,0,303.008403
41,0,2,1,2,1,0,0,270.61,3,0,...,4,2019,7,7,1994,0,0,1,0,347.300580
92,1,5,1,1,1,0,0,343.30,2,0,...,1,2017,15,1,1992,0,0,1,0,313.693628
178,0,23,1,3,1,0,0,401.42,3,0,...,11,2016,24,1,1996,0,0,1,0,532.249163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15740,1,4,2,2,1,0,1,287.12,3,1,...,1,2017,9,5,2008,0,0,1,0,271.486565
15757,1,2,1,1,1,0,1,338.14,3,1,...,9,2018,1,2,2001,0,1,0,0,328.238133
15762,0,5,1,2,2,0,1,260.73,3,0,...,12,2018,2,9,1993,0,0,1,0,404.580446
15771,1,3,1,1,1,0,1,302.09,3,1,...,12,2016,11,9,1991,0,0,1,0,476.820043


In [47]:
# BEFORE FITTING, COLUMNS MUST BE IN THE SAME ORDER

y_pred_HALC = rf_HALC.predict(df_has_LC)

In [48]:
len(y_pred_HALC)

905

In [49]:
# initializing HALC column
df['HALC'] = 0

In [50]:
# IMPORTANT: in the instance that LC is not 0, go to the HALC column (same row), and change 0 into the value of the respective y_pred_HALC index
df.loc[df['LC'] != 0, 'HALC'] = y_pred_HALC

In [51]:
np.mean(df['HALC'])

38.73580581637519

In [52]:
np.mean(df['LC'])

19.834369427714513

In [53]:
df[['LC', 'HALC']].to_csv('LC_HALC_predictions',index=False)

In [54]:
df

,is_channel_broker,total_pol_year,total_pol_held,max_policies,max_products,canceled_policies,is_halfyearly,net_premium,risk_type,is_urban,...,next_renewal_year,license_issue_date_day,license_issue_date_month,license_issue_date_year,is_youngin,is_adult,is_middleaged,is_old,LC,HALC
0,0,2,2,2,1,1,0,240.76,3,1,...,2019,3,2,2011,0,1,0,0,0.0,0.0
1,0,2,1,1,1,1,1,367.97,3,0,...,2017,12,8,1966,0,0,0,1,0.0,0.0
2,0,1,4,4,2,0,0,291.90,3,0,...,2019,2,9,1977,0,0,1,0,0.0,0.0
3,1,5,1,1,1,0,0,303.28,2,1,...,2019,29,10,1980,0,0,1,0,0.0,0.0
4,0,3,1,1,1,1,0,333.30,3,0,...,2018,2,8,2006,0,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15782,0,8,1,2,1,1,0,315.11,3,0,...,2018,7,12,1984,0,0,1,0,0.0,0.0
15783,1,3,2,2,1,0,0,207.11,3,0,...,2018,12,7,1990,0,0,1,0,0.0,0.0
15784,1,5,1,2,1,1,0,324.51,3,1,...,2019,10,7,2006,0,0,0,1,0.0,0.0
15785,1,3,1,1,1,0,1,432.43,3,1,...,2019,23,4,2007,0,0,1,0,0.0,0.0


In [55]:
rf_CS = joblib.load('predict_CS.pkl')

In [56]:
# y_pred_CS = rf_CS.predict(df)
y_pred_CS = rf_CS.predict_proba(df)[:, 1]

In [57]:
df['CS'] = y_pred_CS

In [58]:
# df['CS'].value_counts(

### EXPORTING PREDICTIONS INTO A CSV 

In [60]:
df[['LC', 'HALC', 'CS']].to_csv("group_6_predictions.csv", index=False)